In [1]:
import pandas as pd
import numpy as np
from m4_feats_functions import *
from m7_utils import *
from m5_models import *
from m3_model_params import lgb_params_2 as lgbm_params
from m3_model_params import xgb_params_2 as xgb_params
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import PowerTransformer, MinMaxScaler, StandardScaler
from m3_model_params import non_important_feats

In [2]:
INPUT_DIR = 'kaggle/input/linking-writing-processes-to-writing-quality'
FEAT_STORE_DIR = 'feat_store_combined'
train_logs = pd.read_csv(f'{INPUT_DIR}/train_logs.csv')
train_scores = pd.read_csv(f'{INPUT_DIR}/train_scores.csv')
test_logs = pd.read_csv(f'{INPUT_DIR}/test_logs.csv')
ss_df = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')

logs = pd.concat([train_logs, test_logs], axis=0)

In [3]:
train_feats = pd.read_pickle('feature_store/base_feats/train_base_feats_2.pkl')


In [3]:
train_ids = train_logs.id
test_ids = test_logs.id

logs = pd.concat([train_logs, test_logs], axis=0)
logs = normalise_up_down_times(logs)

train_logs = normalise_up_down_times(train_logs)
test_logs = normalise_up_down_times(test_logs)

preprocessor = Preprocessor(seed=42)
train_feats = preprocessor.make_feats(train_logs)
test_feats = preprocessor.make_feats(test_logs)

essays = getEssays(logs)
sent_feats = compute_sentence_aggregations(essays)
par_feats = compute_paragraph_aggregations(essays)
word_feats = create_word_length_features(essays, 'essay', 'id', 'essay_words')

train_sent = sent_feats[sent_feats['id'].isin(train_ids)]
train_par = par_feats[par_feats['id'].isin(train_ids)]
train_words = word_feats[word_feats['id'].isin(train_ids)]
test_sent = sent_feats[sent_feats['id'].isin(test_ids)]
test_par = par_feats[par_feats['id'].isin(test_ids)]
test_words = word_feats[word_feats['id'].isin(test_ids)]

train_vector, test_vector = countvectorize_one_one(train_logs, test_logs, train_feats, test_feats)

train_feats = train_feats.merge(train_sent, how='left', on='id')
train_feats = train_feats.merge(train_par, how='left', on='id')
train_feats = train_feats.merge(train_words, how='left', on='id')
train_feats = train_feats.merge(train_vector, how='left', on='id')

test_feats = test_feats.merge(test_sent, on='id', how='left')
test_feats = test_feats.merge(test_par, on='id', how='left')
test_feats = test_feats.merge(test_words, on='id', how='left')
test_feats = test_feats.merge(test_vector, on='id', how='left')

train_feats.to_pickle('feat_store_hybrid/train_super.pkl')
test_feats.to_pickle('feat_store_hybrid/test_super.pkl')

Engineering time data
Engineering cursor position data
Engineering word count data
Engineering statistical summaries for features


100%|██████████| 33/33 [01:25<00:00,  2.60s/it, column=word_count_change100, method=kurt]         


Engineering activity counts data


100%|██████████| 2471/2471 [00:00<00:00, 13702.42it/s]
/root/miniconda3/envs/lrp/lib/python3.10/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Engineering event counts data


...

Engineering punctuation counts data


100%|██████████| 3/3 [00:00<00:00, 36792.14it/s]
/root/Projects/Kaggle/linking-writing/m4_feats_functions.py:358: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
/root/Projects/Kaggle/linking-writing/m4_feats_functions.py:359: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
/root/Projects/Kaggle/linking-writing/m4_feats_functions.py:360: PerformanceWarning: DataFrame is highly fragment

Engineering input words data
Engineering ratios data


100%|██████████| 2474/2474 [00:04<00:00, 590.49it/s]


(2474, 27) (2474,)


In [3]:
train_ids = train_logs.id
test_ids = test_logs.id

train_feats = pd.read_pickle('feature_store/base_feats/train_base_feats_2.pkl')
test_feats = pd.read_pickle('feature_store/base_feats/test_base_feats_2.pkl')


# train_feats = train_feats.drop(columns=non_important_feats)
# test_feats = test_feats.drop(columns=test_feats.columns.intersection(non_important_feats))
train_feats = preprocess_feats(train_feats, PowerTransformer('yeo-johnson'))
test_feats = preprocess_feats(test_feats, PowerTransformer('yeo-johnson'))
train_feats = train_feats.merge(train_scores, on='id', how='left')

n_repeats = 5
n_splits = 10
alpha = 450

ridge_params = {'alpha': alpha}  # Create a dictionary with alpha
print(f'Alpha {alpha}')
_, _, ridge_oof_preds, _ = ridge_cv_pipeline(train_feats, test_feats, ridge_params, seed=42, n_repeats=n_repeats, n_splits=n_splits)

train_ids = train_logs.id
test_ids = test_logs.id

train_feats = pd.read_pickle('feature_store/base_feats/train_base_feats_2.pkl')
test_feats = pd.read_pickle('feature_store/base_feats/test_base_feats_2.pkl')

#('action_time_gap', 'count_vectorized', 'action_time_gap_by_acti', 'wc_chage')
# action_gap = pd.read_pickle('feature_store/train/train_action_time_gap.pkl')
# action_gap_act = pd.read_pickle('feature_store/train/train_action_time_gap_by_acti.pkl')
# wc_change = pd.read_pickle('feature_store/train/train_wc_chage.pkl')
# 
# ts_action_gap = pd.read_pickle('feature_store/test/test_action_time_gap.pkl')
# ts_action_gap_act = pd.read_pickle('feature_store/test/test_action_time_gap_by_acti.pkl')
# ts_wc_change = pd.read_pickle('feature_store/test/test_wc_chage.pkl')

# train_feats = train_feats.merge(action_gap, on='id', how='left')
# train_feats = train_feats.merge(action_gap_act, on='id', how='left')
# train_feats = train_feats.merge(wc_change, on='id', how='left')
# 
# test_feats = test_feats.merge(ts_action_gap, on='id', how='left')
# test_feats = test_feats.merge(ts_action_gap_act, on='id', how='left')
# test_feats = test_feats.merge(ts_wc_change, on='id', how='left')

tr_vector = pd.read_pickle('feature_store/train/train_count_vectorized.pkl')
ts_vector = pd.read_pickle('feature_store/test/test_count_vectorized.pkl')
train_feats = train_feats.merge(tr_vector, on='id', how='left')
test_feats = test_feats.merge(ts_vector, on='id', how='left')


train_feats = train_feats.merge(train_scores, on='id', how='left')

_, oof_1, rmse, model1 = xgb_cv_pipeline(train_feats=train_feats, 
                                        test_feats=test_feats, 
                                        xgb_params=xgb_params, 
                                        seed=42, 
                                        n_repeats=n_repeats, 
                                        n_splits=n_splits)
                                        
_, oof_2, rmse, model1 = cv_pipeline(train_feats, 
                                     test_feats, 
                                     lgbm_params, 
                                     lgbm_params['boosting_type'],
                                     seed = 42,
                                     n_repeats= n_repeats,
                                     n_splits = n_splits)

blend = pd.concat([oof_1, oof_2, ridge_oof_preds], axis=0)
blend_scores = blend.groupby(['id','score'])['prediction'].mean().reset_index()
blend_rmse = mean_squared_error(blend_scores['score'], blend_scores['prediction'], squared=False)
print(f'Blend RMSE {blend_rmse:.6f}')

/root/miniconda3/envs/lrp/lib/python3.10/site-packages/numpy/core/_methods.py:239: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)


Alpha 450
Ridge Average RMSE over 50 folds: 0.621465


Iterations:   0%|          | 0/5 [00:00<?, ?it/s]/root/miniconda3/envs/lrp/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [23:00:01] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
Iterations: 100%|██████████| 5/5 [01:44<00:00, 20.86s/it]


XGB Average RMSE over 50 folds: 0.600903
LGBM Average RMSE over 50 folds: 0.601095
Mean RMSE of all iterations: 0.612733
Blend RMSE 0.597671


In [3]:
train_ids = train_logs.id
test_ids = test_logs.id

train_feats = pd.read_pickle('feat_store_hybrid/train_super.pkl')
test_feats = pd.read_pickle('feat_store_hybrid/test_feats.pkl')

train_feats = train_feats.drop(columns=non_important_feats)
test_feats = test_feats.drop(columns=test_feats.columns.intersection(non_important_feats))

# train_feats = preprocess_feats(train_feats, PowerTransformer('yeo-johnson'))
train_feats = train_feats.merge(train_scores, on='id', how='left')

n_repeats=5
n_splits=6

_, oof_1, rmse, model1 = xgb_cv_pipeline(train_feats=train_feats, 
                                        test_feats=test_feats, 
                                        xgb_params=xgb_params, 
                                        seed=42, 
                                        n_repeats=n_repeats, 
                                        n_splits=n_splits)
                                        
_, oof_2, rmse, model1 = cv_pipeline(train_feats, 
                                     test_feats, 
                                     lgbm_params, 
                                     lgbm_params['boosting_type'],
                                     seed = 42,
                                     n_repeats= n_repeats,
                                     n_splits = n_splits)

blend = pd.concat([oof_1, oof_2], axis=0)
blend_scores = blend.groupby(['id','score'])['prediction'].mean().reset_index()
blend_rmse = mean_squared_error(blend_scores['score'], blend_scores['prediction'], squared=False)
print(f'Blend RMSE {blend_rmse:.6f}')

Iterations:   0%|          | 0/5 [00:00<?, ?it/s]/root/miniconda3/envs/lrp/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [09:59:07] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
Iterations: 100%|██████████| 5/5 [00:54<00:00, 10.87s/it]


XGB Average RMSE over 30 folds: 0.605156
LGBM Average RMSE over 30 folds: 0.610728
Blend RMSE 0.605737


In [33]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.inspection import permutation_importance

def run_lgb_model_(model, X_train, y_train, X_valid, y_valid, X_test, boosting_type):
    if boosting_type != 'dart':
        model.fit(X_train, y_train, 
                  eval_set=[(X_valid, y_valid)], 
                  callbacks=[lgb.early_stopping(250, first_metric_only=True, verbose=False)])
    else:
        model.fit(X_train, y_train)  # No early stopping for DART

    valid_predictions = model.predict(X_valid, num_iteration=model.best_iteration_)
    test_predictions = model.predict(X_test, num_iteration=model.best_iteration_)
    return valid_predictions, test_predictions

def run_lgb_cv_(train_feats, test_feats, train_cols, target_col, lgb_params, boosting_type, seed, n_repeats, n_splits):
    oof_results = pd.DataFrame(columns=['id', 'score', 'prediction'])

    X = train_feats[train_cols]
    y = train_feats[target_col].values
    X_test = test_feats[train_cols]

    models = []
    for i in range(n_repeats):
        skf = KFold(n_splits=n_splits, shuffle=True, random_state=seed + i)

        for train_idx, valid_idx in skf.split(X, y):
            X_train, y_train = X.iloc[train_idx], y[train_idx]
            X_valid, y_valid = X.iloc[valid_idx], y[valid_idx]

            model_lgb = lgb.LGBMRegressor(**lgb_params, verbose=-1, random_state=seed)
            valid_preds_lgb, test_preds_lgb = run_lgb_model_(model = model_lgb,
                                               X_train=X_train, y_train=y_train, 
                                               X_valid=X_valid, y_valid=y_valid, 
                                               X_test=X_test, boosting_type=boosting_type)
        
            tmp_df = train_feats.loc[valid_idx][['id','score']]
            tmp_df['prediction'] = valid_preds_lgb
            oof_results = pd.concat([oof_results, tmp_df])
            models.append(model_lgb)

    avg_preds = oof_results.groupby(['id', 'score'])['prediction'].mean().reset_index()
    rmse = mean_squared_error(avg_preds['score'], avg_preds['prediction'], squared=False)
    print(f"LGBM Average RMSE over {n_repeats * n_splits} folds: {rmse:.6f}")
    
    return models, oof_results, rmse

def cv_pipeline_(train_feats, test_feats, lgb_params, boosting_type, seed=42, n_repeats=5, n_splits=10):
    target_col = 'score'
    drop_cols = ['id']
    train_cols = [col for col in train_feats.columns if col not in [target_col] + drop_cols]

    missing_cols = [col for col in train_cols if col not in test_feats.columns]
    missing_cols_df = pd.DataFrame({col: np.nan for col in missing_cols}, index=test_feats.index)
    test_feats = pd.concat([test_feats, missing_cols_df], axis=1)

    models, oof_preds, rmse = run_lgb_cv_(train_feats=train_feats, test_feats=test_feats, 
                                         train_cols=train_cols, target_col=target_col, 
                                         lgb_params=lgb_params, boosting_type=boosting_type,
                                         seed=seed, n_repeats=n_repeats, n_splits=n_splits)

    # Calculate permutation feature importance
    importance_scores = np.zeros((len(models), len(train_cols)))
    for i, model in enumerate(models):
        result = permutation_importance(model, train_feats[train_cols], train_feats[target_col], n_repeats=30, random_state=seed)
        importance_scores[i, :] = result.importances_mean

    importance_scores_mean = np.mean(importance_scores, axis=0)
    feature_importance = pd.DataFrame({'feature': train_cols, 'importance': importance_scores_mean})
    feature_importance.sort_values(by='importance', ascending=False, inplace=True)

    print("Number of models:", len(models))
    print("Train columns:", train_cols)

    if np.all(importance_scores == 0):
        print("Warning: All importance scores are zero. Check your data and model.")

    importance_scores_mean = np.mean(importance_scores, axis=0)
    feature_importance = pd.DataFrame({'feature': train_cols, 'importance': importance_scores_mean})
    feature_importance.sort_values(by='importance', ascending=False, inplace=True)

    return models, oof_preds, rmse, feature_importance

_,_,_,feature_importance = cv_pipeline_(train_feats, test_feats, lgbm_params, 'gbdt')

LGBM Average RMSE over 50 folds: 0.608614
Number of models: 50
Train columns: ['event_id_max', 'up_time_max', 'action_time_max', 'action_time_min', 'action_time_mean', 'action_time_std', 'action_time_quantile', 'action_time_sem', 'action_time_sum', 'action_time_skew', 'action_time_kurt', 'activity_nunique', 'down_event_nunique', 'up_event_nunique', 'text_change_nunique', 'cursor_position_nunique', 'cursor_position_max', 'cursor_position_quantile', 'cursor_position_sem', 'cursor_position_mean', 'word_count_nunique', 'word_count_max', 'word_count_quantile', 'word_count_sem', 'word_count_mean', 'action_time_gap1_max', 'action_time_gap1_min', 'action_time_gap1_mean', 'action_time_gap1_std', 'action_time_gap1_quantile', 'action_time_gap1_sem', 'action_time_gap1_sum', 'action_time_gap1_skew', 'action_time_gap1_kurt', 'cursor_position_change1_max', 'cursor_position_change1_mean', 'cursor_position_change1_std', 'cursor_position_change1_quantile', 'cursor_position_change1_sem', 'cursor_position

In [40]:
non_important_feats = feature_importance[feature_importance['importance']==0]['feature']

In [43]:
non_important_feats

373                              tok_21
372                              tok_20
366                              tok_14
3                       action_time_min
367                              tok_15
375                              tok_23
368                              tok_16
374                              tok_22
369                              tok_17
376                              tok_24
377                              tok_25
370                              tok_18
371                              tok_19
258                   up_event_12_count
37     cursor_position_change1_quantile
95          word_count_change3_quantile
261                   up_event_15_count
256                   up_event_10_count
248                    up_event_2_count
245                 down_event_15_count
120         word_count_change5_quantile
313                      low_sent_count
112    cursor_position_change5_quantile
92               word_count_change3_max
351                     essay_words_min


In [ ]:
##### RIDGE REGRESSION FOR BLENDING
train_feats = pd.read_pickle('feat_store_hybrid/train_super.pkl')
test_feats = pd.read_pickle('feat_store_hybrid/test_super.pkl')

train_feats = train_feats.drop(columns=non_important_feats)
test_feats = test_feats.drop(columns=test_feats.columns.intersection(non_important_feats)) #yj 0.621593
train_feats = preprocess_feats(train_feats, PowerTransformer('yeo-johnson')) 
test_feats = preprocess_feats(test_feats, PowerTransformer('yeo-johnson'))
# train_feats = preprocess_feats(train_feats, PowerTransformer('yeo-johnson'))
train_feats = train_feats.merge(train_scores, on='id', how='left')

n_repeats = 5
n_splits = 6
alpha = 450
param_grid = list(np.arange(400, 900, 50))

for alpha in param_grid:
    ridge_params = {'alpha': alpha}  # Create a dictionary with alpha
    print(f'Alpha {alpha}')
    _, _, ridge_oof_preds, _ = ridge_cv_pipeline(train_feats, test_feats, ridge_params, seed=42, n_repeats=n_repeats, n_splits=n_splits)

Alpha 400
Ridge Average RMSE over 30 folds: 0.621593
Alpha 450
Ridge Average RMSE over 30 folds: 0.621564
Alpha 500
Ridge Average RMSE over 30 folds: 0.621566
Alpha 550
Ridge Average RMSE over 30 folds: 0.621593
Alpha 600
Ridge Average RMSE over 30 folds: 0.621640
Alpha 650
Ridge Average RMSE over 30 folds: 0.621703
Alpha 700
Ridge Average RMSE over 30 folds: 0.621779
Alpha 750
Ridge Average RMSE over 30 folds: 0.621867
Alpha 800
Ridge Average RMSE over 30 folds: 0.621963
Alpha 850
Ridge Average RMSE over 30 folds: 0.622068
